In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

import re

def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        cur_word=words[0].lower()
        words_pair=[]
        for word in words[1:]:
            words_pair.append(cur_word+"_"+word.lower())
            cur_word=word.lower()
        return words_pair
    except ValueError as e:
        return []

wiki = sc.textFile("/data/wiki/en_articles_part/articles-part", 16).flatMap(parse_article).map(lambda x:(x,1)).filter(lambda x:x[0].startswith("narodnaya_"))
wiki_count=wiki.reduceByKey(lambda x,y:x+y).sortByKey()                                                                                             
result = wiki_count.collect()
sc.stop()

In [2]:
# sc.stop()
for word,count in result:
    print word,count

narodnaya_gazeta 1
narodnaya_volya 9
